<a href="https://colab.research.google.com/github/kiselpd/HW5_ML/blob/main/price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#for my GoogleDisk
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/GoogleColab/HW5

Mounted at /content/drive
/content/drive/MyDrive/GoogleColab/HW5


In [81]:
%matplotlib inline
import pandas as pd
import numpy as np

In [82]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [83]:
train_df = pd.read_csv('Train.csv')

In [84]:
train_df

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,0,2011-1,560,NaN,2,59,3,0,30.0,1.0,5,0,1,0,0,0,0,0,0,0,0,0,0,0,4510000
1,1,2011-1,667,NaN,10,50,2,1,25.0,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,13231000
2,2,2011-1,90,0.0,1,48,2,0,25.0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2008000
3,3,2011-1,94,1.0,3,62,3,1,30.0,NaN,3,0,1,0,0,0,0,0,0,0,0,0,0,0,12680000
4,4,2011-1,232,0.0,3,60,3,0,25.0,NaN,3,0,1,0,0,0,0,0,0,0,0,0,0,0,3335000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,2012-3,401,0.0,5,30,1,0,30.0,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1697000
99996,99996,2012-3,59,0.0,4,34,1,0,30.0,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,3620000
99997,99997,2012-3,253,NaN,3,52,2,0,5.0,NaN,4,0,1,0,0,0,0,0,0,0,0,0,0,0,6712000
99998,99998,2012-3,305,1.0,3,181,5,1,5.0,NaN,3,0,0,0,0,0,0,0,0,0,0,0,0,0,20835000


In [85]:
import datetime as dt

In [86]:
train_df["date"] = pd.to_datetime(train_df["date"]).map(dt.datetime.toordinal)
train_df = train_df.drop(columns = ["id"])

In [87]:
(train_df.isna().any()).to_numpy() #NaN

array([False, False,  True, False, False, False, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [88]:
from sklearn.impute import KNNImputer

In [89]:
train_columns = train_df.columns

In [90]:
train_columns

Index(['date', 'street_id', 'build_tech', 'floor', 'area', 'rooms', 'balcon',
       'metro_dist', 'g_lift', 'n_photos', 'kw1', 'kw2', 'kw3', 'kw4', 'kw5',
       'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12', 'kw13', 'price'],
      dtype='object')

In [91]:
imputer = KNNImputer(n_neighbors = 20)
train_df = imputer.fit_transform(train_df)

In [92]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

In [93]:
train_df = pd.DataFrame(train_df, columns = train_columns)

In [94]:
df = pd.DataFrame(columns = train_columns)

In [95]:
df = df.drop(columns = ["price"])

In [96]:
df = df.assign( LiR= "", RFR = "", BR = "")

In [97]:
train_df["build_tech"] = train_df["build_tech"].round()
train_df["g_lift"] = train_df["g_lift"].round()

In [98]:
df

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,LiR,RFR,BR


In [99]:
kfold = KFold(n_splits = 10, shuffle = False)

In [100]:
for train, test in kfold.split(train_df):
  X_train = train_df.loc[train].drop(columns = ["price"])
  y_train = (train_df.loc[train])["price"]

  X_test = train_df.loc[test].drop(columns = ["price"])
  y_test = (train_df.loc[test])["price"]

  #LinearRegression
  LiR = LinearRegression()
  LiR.fit(X_train, y_train)
  LiR_pred = LiR.predict(X_test)
  print("LIR", mean_absolute_error(y_test, LiR_pred))

  #RandomForestRegressor
  RFR = RandomForestRegressor(n_estimators = 200, min_samples_split = 4)
  RFR.fit(X_train, y_train)
  RFR_pred = RFR.predict(X_test)
  print("RFR", mean_absolute_error(y_test, RFR_pred))

  #BayesianRidge
  BR = BayesianRidge()
  BR.fit(X_train, y_train)
  BR_pred = BR.predict(X_test)
  print("BR", mean_absolute_error(y_test, BR_pred))

  stack = np.hstack([X_test, LiR_pred.reshape(-1, 1), RFR_pred.reshape(-1, 1), BR_pred.reshape(-1, 1)])
  stack = pd.DataFrame(stack, columns = df.columns)
  df = df.append(stack)



LIR 1670728.123930079
RFR 1507127.7459641302
BR 1670812.4151110626
LIR 1829944.514740188
RFR 1649445.2542469183
BR 1830013.858512838
LIR 1685978.0116750728
RFR 1502346.8040493508
BR 1685341.6898220428
LIR 1641838.4327686797
RFR 1446923.8981847137
BR 1641787.91041197
LIR 1659545.4031369998
RFR 1494466.016685319
BR 1659473.8284409042
LIR 1773467.045271355
RFR 1580636.4371595113
BR 1773566.891379641
LIR 1825050.6135481666
RFR 1633218.493990518
BR 1825260.75643837
LIR 1966685.5246766068
RFR 1765991.8978082184
BR 1966756.307808704
LIR 1981847.9788260688
RFR 1783860.3815593652
BR 1982024.763991539
LIR 1864252.8648920697
RFR 1693420.556074001
BR 1863745.396205681


In [101]:
df

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,LiR,RFR,BR
0,734138.0,560.0,1.0,2.0,59.0,3.0,0.0,30.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.134567e+06,5.195605e+06,5.133928e+06
1,734138.0,667.0,1.0,10.0,50.0,2.0,1.0,25.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.685239e+06,6.325080e+06,4.684658e+06
2,734138.0,90.0,0.0,1.0,48.0,2.0,0.0,25.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.715273e+06,3.439344e+06,2.716396e+06
3,734138.0,94.0,1.0,3.0,62.0,3.0,1.0,30.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.306898e+06,4.686444e+06,5.307053e+06
4,734138.0,232.0,0.0,3.0,60.0,3.0,0.0,25.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.864496e+06,3.735447e+06,3.866749e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,734563.0,401.0,0.0,5.0,30.0,1.0,0.0,30.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.480851e+06,1.767082e+06,1.480733e+06
9996,734563.0,59.0,0.0,4.0,34.0,1.0,0.0,30.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.243121e+06,2.466220e+06,2.242948e+06
9997,734563.0,253.0,1.0,3.0,52.0,2.0,0.0,5.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.271346e+06,6.873465e+06,7.269690e+06
9998,734563.0,305.0,1.0,3.0,181.0,5.0,1.0,5.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.769515e+07,2.905917e+07,2.768997e+07


In [102]:
GradientBoostingRegressor
GBR = GradientBoostingRegressor()
GBR.fit(df, train_df["price"])

GradientBoostingRegressor()

In [63]:
test_df = pd.read_csv('Test.csv')

In [64]:
id = test_df["id"]

In [65]:
test_df["date"] = pd.to_datetime(test_df["date"]).map(dt.datetime.toordinal)
test_df = test_df.drop(columns = ["id"])

In [66]:
imputer = KNNImputer(n_neighbors = 20)
test_df = imputer.fit_transform(test_df)

In [67]:
test_df = pd.DataFrame(test_df, columns = (train_df.drop(columns = ["price"]).columns))

In [68]:
test_df["build_tech"] = test_df["build_tech"].round()
test_df["g_lift"] = test_df["g_lift"].round()

In [103]:
test_df

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13
0,734563.0,459.0,1.0,1.0,60.0,3.0,1.0,30.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,734563.0,344.0,1.0,10.0,52.0,2.0,1.0,20.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,734563.0,585.0,0.0,4.0,54.0,3.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,734563.0,494.0,1.0,2.0,52.0,2.0,1.0,25.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,734563.0,622.0,1.0,9.0,60.0,3.0,1.0,15.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,735112.0,613.0,1.0,2.0,34.0,1.0,1.0,25.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,735112.0,167.0,1.0,10.0,61.0,3.0,1.0,30.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,735112.0,391.0,1.0,9.0,34.0,1.0,1.0,30.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,735112.0,21.0,0.0,7.0,58.0,3.0,0.0,30.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
#LinearRegression
LiR_test_pred = LiR.predict(test_df)
#RandomForestRegressor
RFR_test_pred = RFR.predict(test_df)
#BayesianRidge
BR_test_pred = BR.predict(test_df)

In [106]:
test_stack = np.hstack([test_df, LiR_test_pred.reshape(-1, 1), RFR_test_pred.reshape(-1, 1), BR_test_pred.reshape(-1, 1)])

In [72]:
#test_stack = test_stack.astype(np.int32)

In [107]:
test_stack

array([[7.34563000e+05, 4.59000000e+02, 1.00000000e+00, ...,
        5.65921878e+06, 6.70867807e+06, 5.65832518e+06],
       [7.34563000e+05, 3.44000000e+02, 1.00000000e+00, ...,
        6.72416475e+06, 7.18986638e+06, 6.72317582e+06],
       [7.34563000e+05, 5.85000000e+02, 0.00000000e+00, ...,
        2.41209875e+06, 2.32550260e+06, 2.41418560e+06],
       ...,
       [7.35112000e+05, 3.91000000e+02, 1.00000000e+00, ...,
        4.92233614e+06, 4.74867766e+06, 4.92006973e+06],
       [7.35112000e+05, 2.10000000e+01, 0.00000000e+00, ...,
        4.99374657e+06, 3.45931234e+06, 4.99549125e+06],
       [7.35112000e+05, 4.09000000e+02, 0.00000000e+00, ...,
        3.43199585e+06, 2.84976309e+06, 3.43194513e+06]])

In [108]:
df = pd.DataFrame(columns = test_df.columns)
df = df.assign(LiR = "", RFR = "", BR = "")
test_stack = pd.DataFrame(test_stack, columns = df.columns)
df = df.append(test_stack)
df

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,LiR,RFR,BR
0,734563.0,459.0,1.0,1.0,60.0,3.0,1.0,30.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.659219e+06,6.708678e+06,5.658325e+06
1,734563.0,344.0,1.0,10.0,52.0,2.0,1.0,20.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.724165e+06,7.189866e+06,6.723176e+06
2,734563.0,585.0,0.0,4.0,54.0,3.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.412099e+06,2.325503e+06,2.414186e+06
3,734563.0,494.0,1.0,2.0,52.0,2.0,1.0,25.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.829213e+06,4.665804e+06,5.827259e+06
4,734563.0,622.0,1.0,9.0,60.0,3.0,1.0,15.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.165829e+06,7.292555e+06,6.165873e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,735112.0,613.0,1.0,2.0,34.0,1.0,1.0,25.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.598794e+06,3.236476e+06,4.595485e+06
99996,735112.0,167.0,1.0,10.0,61.0,3.0,1.0,30.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.069178e+06,4.490159e+06,7.069279e+06
99997,735112.0,391.0,1.0,9.0,34.0,1.0,1.0,30.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.922336e+06,4.748678e+06,4.920070e+06
99998,735112.0,21.0,0.0,7.0,58.0,3.0,0.0,30.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.993747e+06,3.459312e+06,4.995491e+06


In [109]:
GBR_test_pred = GBR.predict(df)

In [110]:
GBR_test_pred

array([6421236.28810211, 7169827.9292281 , 2363926.19643406, ...,
       4617017.09231574, 3401167.30144066, 2880518.97655419])

In [111]:
result = np.hstack([(id.to_numpy()).reshape(-1, 1), GBR_test_pred.reshape(-1, 1)])
result = pd.DataFrame(result, columns = ["id", "price"])
result = result.astype(np.int32)

In [112]:
result

,id,price
0,100000,6421236
1,100001,7169827
2,100002,2363926
3,100003,4588511
4,100004,7116862
...,...,...
99995,199995,3545945
99996,199996,4399194
99997,199997,4617017
99998,199998,3401167


In [114]:
result.to_csv("answer.csv", index = False)